In [0]:
!pip install -q keras

In [8]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
import tensorflow as tf

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 113597 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
from keras.models import model_from_json

In [9]:


!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"

mkdir: cannot create directory ‘drive’: File exists
color_tensorflow_real_mode.h5  model.json  result  Train
colourize.ipynb		       output	   Test


In [0]:
X = []
for filename in os.listdir('drive/Colab Notebooks/Train/'):
    X.append(img_to_array(load_img('drive/Colab Notebooks/Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

In [15]:
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

Instructions for updating:
Colocations handled automatically by placer.
225214464/225209952 [==============================] - 4s 0us/step


In [0]:
embed_input = Input(shape=(1000,))
#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [0]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

In [0]:
# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 20

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [54]:
tensorboard = TensorBoard(log_dir="drive/Colab Notebooks/output")
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=100, steps_per_epoch=20)

Epoch 1/100
20/20 [==============================] - 34s 2s/step - loss: 0.0062
Epoch 2/100
20/20 [==============================] - 31s 2s/step - loss: 0.0055
Epoch 3/100
20/20 [==============================] - 31s 2s/step - loss: 0.0051
Epoch 4/100
20/20 [==============================] - 31s 2s/step - loss: 0.0049
Epoch 5/100
20/20 [==============================] - 31s 2s/step - loss: 0.0048
Epoch 6/100
20/20 [==============================] - 31s 2s/step - loss: 0.0045
Epoch 7/100
20/20 [==============================] - 31s 2s/step - loss: 0.0043
Epoch 8/100
20/20 [==============================] - 31s 2s/step - loss: 0.0041
Epoch 9/100
20/20 [==============================] - 31s 2s/step - loss: 0.0037
Epoch 10/100
20/20 [==============================] - 31s 2s/step - loss: 0.0034
Epoch 11/100
20/20 [==============================] - 31s 2s/step - loss: 0.0031
Epoch 12/100
20/20 [==============================] - 31s 2s/step - loss: 0.0031
Epoch 13/100
20/20 [=================

In [29]:
tensorboard = TensorBoard(log_dir="drive/Colab Notebooks/output")
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
20/20 [==============================] - 35s 2s/step - loss: 0.0399
Epoch 2/10
20/20 [==============================] - 30s 2s/step - loss: 0.0056
Epoch 3/10
20/20 [==============================] - 31s 2s/step - loss: 0.0054
Epoch 4/10
20/20 [==============================] - 30s 2s/step - loss: 0.0050
Epoch 5/10
20/20 [==============================] - 30s 2s/step - loss: 0.0049
Epoch 6/10
20/20 [==============================] - 30s 2s/step - loss: 0.0047
Epoch 7/10
20/20 [==============================] - 31s 2s/step - loss: 0.0045
Epoch 8/10
20/20 [==============================] - 31s 2s/step - loss: 0.0045
Epoch 9/10
20/20 [==============================] - 31s 2s/step - loss: 0.0043
Epoch 10/10
20/20 [==============================] - 31s 2s/step - loss: 0.0040


In [30]:
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=40)

Epoch 1/10
40/40 [==============================] - 61s 2s/step - loss: 0.0037
Epoch 2/10
40/40 [==============================] - 61s 2s/step - loss: 0.0034
Epoch 3/10
40/40 [==============================] - 61s 2s/step - loss: 0.0031
Epoch 4/10
40/40 [==============================] - 61s 2s/step - loss: 0.0025
Epoch 5/10
40/40 [==============================] - 61s 2s/step - loss: 0.0022
Epoch 6/10
40/40 [==============================] - 64s 2s/step - loss: 0.0019
Epoch 7/10
40/40 [==============================] - 62s 2s/step - loss: 0.0017
Epoch 8/10
40/40 [==============================] - 62s 2s/step - loss: 0.0015
Epoch 9/10
40/40 [==============================] - 64s 2s/step - loss: 0.0013
Epoch 10/10
40/40 [==============================] - 62s 2s/step - loss: 0.0012


In [21]:
# load json and create model
json_file = open('drive/Colab Notebooks/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("drive/Colab Notebooks/color_tensorflow_real_mode.h5")
print("Loaded model from disk")

Loaded model from disk


In [0]:
loaded_model.compile(optimizer='adam', loss='mse')

In [0]:
color_me = []
for filename in os.listdir('drive/Colab Notebooks/Test/'):
    color_me.append(img_to_array(load_img('drive/Colab Notebooks/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = 1.0/255*color_me
color_me = gray2rgb(rgb2gray(color_me))
color_me_embed = create_inception_embedding(color_me)
color_me = rgb2lab(color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

In [0]:
#Test model
output = loaded_model.predict([color_me, color_me_embed])
output = output * 128

In [33]:
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("drive/Colab Notebooks/result/img_abc"+str(i)+".png", lab2rgb(cur))

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
